In [4]:
# Fix for issue loading Utils.preprocess_util
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')

import numpy as np
import torch
import torch.nn as nn
import torch.optim  as optim
from torch.autograd import Variable
from Utils.preprocess_util import *
from Utils.visualize import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
X_train,X_valid,X_test,Y_train,Y_valid,Y_test = load_preprocess_eeg_data()


MemoryError: 

In [4]:
class Flatten(nn.Module):
    def forward(self, x):
        a= x.view(x.size(0), -1)
        return a
    
class threed_to_twod(nn.Module):
    def forward(self, x):
        # print(x.shape)
        a = x.reshape(x.shape[0],x.shape[3],x.shape[1])
        # print (a)
        return a

In [5]:
droput = 0
model = nn.Sequential()
model.add_module('conv_across_time',nn.Conv2d(1,40,kernel_size=(1,51),stride = 1))
model.add_module('conv_across_electrodes',nn.Conv2d(40,40,kernel_size=(22,1),stride = 1))
model.add_module('BatchNorm2d',nn.BatchNorm2d(40,momentum=0.1))
model.add_module('Nonlinearity', nn.ReLU())
model.add_module('correct_dimensions',threed_to_twod())
model.add_module('AvgPool2d',nn.AvgPool2d(kernel_size=(135,1),stride = (5,1)))
model.add_module('drop', nn.Dropout(p=droput))
model.add_module('Flatten',Flatten())    
model.add_module('Fc_layer',nn.Linear(2560,10))
torch.nn.init.xavier_uniform_(model.conv_across_time.weight, gain=1)
torch.nn.init.xavier_uniform_(model.conv_across_electrodes.weight, gain=1)


Parameter containing:
tensor([[[[ 0.0354],
          [-0.0201],
          [-0.0546],
          ...,
          [-0.0403],
          [-0.0247],
          [ 0.0454]],

         [[ 0.0531],
          [-0.0061],
          [ 0.0152],
          ...,
          [-0.0487],
          [ 0.0410],
          [ 0.0578]],

         [[ 0.0139],
          [-0.0294],
          [-0.0184],
          ...,
          [-0.0395],
          [ 0.0205],
          [-0.0304]],

         ...,

         [[-0.0119],
          [ 0.0139],
          [-0.0219],
          ...,
          [-0.0280],
          [ 0.0356],
          [-0.0134]],

         [[ 0.0251],
          [ 0.0477],
          [-0.0061],
          ...,
          [-0.0378],
          [-0.0442],
          [ 0.0482]],

         [[-0.0363],
          [ 0.0270],
          [-0.0271],
          ...,
          [-0.0385],
          [ 0.0315],
          [-0.0049]]],


        [[[ 0.0127],
          [-0.0140],
          [ 0.0008],
          ...,
          [ 0.0254],
    

In [6]:
dtype = torch.FloatTensor
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)


In [7]:
# N,C,H,W = 18,1,25,1000
# x = Variable(torch.tensor(X_train.reshape((18,1, 25, 1000))))
x = Variable(torch.tensor(X_train))
y = Variable(torch.tensor(Y_train),requires_grad=False)
dtype = torch.FloatTensor
x.type(dtype)
y.type(dtype)

tensor([3., 3., 3.,  ..., 0., 0., 0.])

In [13]:
'''
for t in range(3):
    y_pred = model( x.float())
    loss = loss_fn(y_pred,y.type(torch.LongTensor))
    print(loss.data)
    model.zero_grad()
    loss.backward()
loss = loss_fn(y_pred,y.type(torch.LongTensor))
'''

'\nfor t in range(3):\n    y_pred = model( x.float())\n    loss = loss_fn(y_pred,y.type(torch.LongTensor))\n    print(loss.data)\n    model.zero_grad()\n    loss.backward()\nloss = loss_fn(y_pred,y.type(torch.LongTensor))\n'

In [ ]:
num_train = X_train.shape[0]
num_valid = X_valid.shape[0]
batch_size = 100
num_epochs = 50
iterations_per_epoch = max(num_train // batch_size, 1)
num_iterations = num_epochs * iterations_per_epoch
epoch = 1
train_loss = []
iterations = []

optimizer = torch.optim.Adam(model.parameters(), lr=0.000001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)   
for t in range(num_iterations):
    batch_mask = np.random.choice(num_train, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = Y_train[batch_mask]
    X_batch_tensor = threeD_to_fourDTensor(X_batch)
    y_batch_tensor = Variable(torch.tensor(y_batch))
    
    y_pred = model( X_batch_tensor.float())
    
    loss = loss_fn(y_pred,y_batch_tensor.type(torch.LongTensor))
    
    train_loss.append(loss.data)
    iterations.append(t)
        
    model.zero_grad()
    loss.backward()
    optimizer.step()
    if(t%10 == 0):
        print('(Iteration %d / %d) loss: %f' % (
                       t + 1, num_iterations, loss.detach().numpy()))
    
    epoch_end = (t + 1) % iterations_per_epoch == 0
    
    if epoch_end:
                epoch += 1
    
    first_it = (t == 0)
    last_it = (t == num_iterations - 1)

    if first_it or last_it or epoch_end:
        X_train_tensor =threeD_to_fourDTensor(X_train[0:100,:,:])
        y_pred_train = model( X_train_tensor.float())
        train_acc = get_accuracy(y_pred_train, Y_train[0:100],
            batch_size=50)
        
        X_valid_tensor = threeD_to_fourDTensor(X_valid[0:50,:,:])
        y_pred_valid = model( X_valid_tensor.float())
        val_acc = get_accuracy(y_pred_valid, Y_valid[0:50],
            batch_size=50)
        print('(Epoch %d / %d) train acc: %f; val_acc: %f' % (
                           epoch, num_epochs, train_acc, val_acc))


(Iteration 1 / 88550) loss: 2.254832
(Epoch 1 / 50) train acc: 0.000000; val_acc: 0.000000
(Iteration 11 / 88550) loss: 2.243098
(Iteration 21 / 88550) loss: 2.247763
(Iteration 31 / 88550) loss: 2.246058
(Iteration 41 / 88550) loss: 2.221097
(Iteration 51 / 88550) loss: 2.230875
(Iteration 61 / 88550) loss: 2.232251


In [9]:

y_pred_valid = model( threeD_to_fourDTensor(X_valid).float())
val_acc = get_accuracy(y_pred_valid, Y_valid,
    batch_size=X_valid.shape[0])
print('validation accuracy:', val_acc)

RuntimeError: $ Torch: not enough memory: you tried to allocate 3GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [23]:
#Calculating the validation accruacy in baaches, since out-of-memory error arises when calculating using the whole set!
datset_size =  X_valid.shape[0]
size_of_batches = int(datset_size/500)

print(datset_size)
print(size_of_batches)

start = 0

valid_accuracies = []
for i in range(500):
    end = start + size_of_batches
    
    y_pred_valid = model( threeD_to_fourDTensor(X_valid[start:end]).float())
    val_acc = get_accuracy(y_pred_valid, Y_valid[start:end],
    batch_size=len(Y_valid[start:end]))
    start = end
    valid_accuracies.append(val_acc)

print('validation accuracy:', np.mean(valid_accuracies))

87250
174
validation accuracy: 37.1183908045977


In [25]:
#Calculating the training accruacy in batches, since out-of-memory error arises when calculating using the whole set!
datset_size =  X_train.shape[0]
size_of_batches = int(datset_size/500)

print(datset_size)
print(size_of_batches)

start=end = 0

train_accuracies = []
for i in range(500):
    end = start + size_of_batches
    
    y_pred_train = model( threeD_to_fourDTensor(X_train[start:end]).float())
    train_acc = get_accuracy(y_pred_train, Y_train[start:end],
    batch_size=len(Y_train[start:end]))
    start = end
    train_accuracies.append(train_acc)

print('train accuracy:', np.mean(train_accuracies))


177125
354
train accuracy: 47.509604519774015


In [26]:
#Calculating the test accruacy in baaches, since out-of-memory error arises when calculating using the whole set!
datset_size =  X_test.shape[0]
size_of_batches = int(datset_size/500)

print(datset_size)
print(size_of_batches)

start = 0

test_accuracies = []
for i in range(500):
    end = start + size_of_batches
    y_pred_test = model( threeD_to_fourDTensor(X_test[start:end]).float())
    test_acc = get_accuracy(y_pred_test, Y_test[start:end],
    batch_size=len(Y_test[start:end]))
    start = end
    test_accuracies.append(test_acc)

print('Test accuracy:', np.mean(test_accuracies))

55375
110
Test accuracy: 35.74


In [20]:
X_test_tensor = threeD_to_fourDTensor(X_test)
y_pred_test = model( X_test_tensor.float())
test_acc = get_accuracy(y_pred_test, Y_test,
    batch_size=X_test.shape[0])
print('test accuracy:', test_acc)

RuntimeError: $ Torch: not enough memory: you tried to allocate 11GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [ ]:
import matplotlib.pyplot as plt
plt.plot(iterations,train_loss)
plt.show();